In [ ]:
import pandas as pd
import importlib

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [ ]:
def load_data_and_apply_strategies(price_path, strategy_map_path):
    # 1. Load price data
    df_prices = pd.read_csv(price_path, parse_dates=['date'])

    # 2. Load asset-strategy mapping
    strat_df = pd.read_csv(strategy_map_path)

    # 3. Prepare DataFrame (MultiIndex)
    df_prices = df_prices.set_index(['date', 'asset']).sort_index()

    # 4. Dynamically load and apply strategies
    df_list = []
    strategy_instances = {}

    for _, row in strat_df.iterrows():
        asset = row['asset']
        strat_class_name = row['strategy']
        module_name = strat_class_name.lower()  # assumes strategy06.py for Strategy06 -- can be adjusted later based on our new naming

        # Load strategy class dynamically
        strategy_module = importlib.import_module(f'strategies.{module_name}')
        strategy_class = getattr(strategy_module, strat_class_name)
        strategy_instance = strategy_class()
        strategy_instances[asset] = strategy_instance

        # Apply strategy to that asset's data
        asset_df = df_prices.xs(asset, level='asset').copy()
        asset_df = strategy_instance.generate_signals(asset_df)
        asset_df['asset'] = asset
        df_list.append(asset_df.reset_index())

    # 5. Combine all assets back into one MultiIndex DataFrame
    full_df = pd.concat(df_list).set_index(['date', 'asset']).sort_index()

    return full_df, strat_df, strategy_instances

In [ ]:
def auto_trade(df, strategy_map, strat_df, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(['date', 'asset'])

    assets = df.index.get_level_values(1).unique()
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {'units': 0, 'entry_price': None} for asset in assets}
    cash = config['starting_cash']
    trade_log = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )

        # Calculate thresholds
        cash_floor = config['cash_floor_pct'] * total_value
        cash_ceiling = config['cash_ceiling_pct'] * total_value
    
        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row['Close']
            signal = row['TotalSignal']

            # Get max allocation for this asset
            ideal_proportion = strat_df.loc[strat_df['asset'] == asset, 'ideal_proportion'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]['units'] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config['buy_pct'] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            if portfolio[asset]['units'] > 0:
                entry_price = portfolio[asset]['entry_price']
                profit_pct = (current_price - entry_price) / entry_price
                if profit_pct >= config['tp_pct'] or profit_pct <= -config['sl_pct']: #IMPLEMENT DYNAMIC TP AND SL
                    proceeds = portfolio[asset]['units'] * current_price * (1 - config['fee'])
                    cash += proceeds
                    trade_log.append({
                        'asset': asset, 'date': date, 'action': 'SELL_TP/SL', #MAYBE SEPARATE TP and SL for LOG PURPOSE
                        'price': current_price, 'units': portfolio[asset]['units']
                    })
                    portfolio[asset] = {'units': 0, 'entry_price': None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]['units'] > 0:
                proceeds = portfolio[asset]['units'] * current_price * (1 - config['fee'])
                cash += proceeds
                trade_log.append({
                    'asset': asset, 'date': date, 'action': 'SELL_SHORT',
                    'price': current_price, 'units': portfolio[asset]['units']
                })
                portfolio[asset] = {'units': 0, 'entry_price': None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                if signal == 2:
                    expected_return = row.get('Expected Return', None)
                    if expected_return is None or expected_return <= 2 * config['fee']:
                        continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0 and (cash - buy_amount) >= cash_floor:
                    fee = config['fee']
                    new_units = (buy_amount * (1 - fee)) / current_price
                    old_units = portfolio[asset]['units']
                    old_price = portfolio[asset]['entry_price']

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price) / total_units
                        if old_units > 0 else current_price
                    ) #update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]['units'] = total_units
                    portfolio[asset]['entry_price'] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append({
                        'asset': asset, 'date': date, 'action': 'BUY_ADD' if old_units > 0 else 'BUY_NEW',
                        'price': current_price,'units': new_units
                    })

        # ---------- GOLD Allocation ---------- only when cash reserve is higher than cash ceiling
        #Limitation: not converting gold to cash real-time if need to buy assets - missing opportunities
        if 'GOLD' in assets and date >= dates[15] and cash > cash_ceiling and (date, 'GOLD') in df.index:
            gold_row = df.loc[(date, 'GOLD')]
            gold_price = gold_row['Close']
            gold_signal = gold_row['TotalSignal']

            # Check TP/SL for gold
            if portfolio['GOLD']['units'] > 0:
                entry_price = portfolio['GOLD']['entry_price']
                profit_pct = (gold_price - entry_price) / entry_price
                if profit_pct >= config['tp_pct'] or profit_pct <= -config['sl_pct']:
                    proceeds = portfolio['GOLD']['units'] * gold_price * (1 - config['fee'])
                    cash += proceeds
                    trade_log.append({
                        'asset': 'GOLD',
                        'date': date,
                        'action': 'SELL_TP/SL_GOLD',
                        'price': gold_price,
                        'units': portfolio['GOLD']['units']
                    })
                    portfolio['GOLD'] = {'units': 0, 'entry_price': None}

            # SELL if signal is SHORT
            if gold_signal == 1 and portfolio['GOLD']['units'] > 0:
                proceeds = portfolio['GOLD']['units'] * gold_price * (1 - config['fee'])
                cash += proceeds
                trade_log.append({
                    'asset': 'GOLD',
                    'date': date,
                    'action': 'SELL_GOLD',
                    'price': gold_price,
                    'units': portfolio['GOLD']['units']
                })
                portfolio['GOLD'] = {'units': 0, 'entry_price': None}

            # BUY gold if LONG signal and cash > ceiling
            elif gold_signal == 2:
                excess_cash = cash - cash_ceiling
                invest_amount = min(excess_cash, 0.10 * total_value) #CAN CHANGE THE 0.10

                if invest_amount > 0 and (cash - invest_amount) >= cash_floor:
                    new_units = (invest_amount * (1 - config['fee'])) / gold_price
                    old_units = portfolio['GOLD']['units']
                    old_price = portfolio['GOLD']['entry_price']

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * gold_price) / total_units
                        if old_units > 0 else gold_price
                    )

                    portfolio['GOLD']['units'] = total_units
                    portfolio['GOLD']['entry_price'] = new_avg_price
                    cash -= invest_amount

                    trade_log.append({
                        'asset': 'GOLD',
                        'date': date,
                        'action': 'BUY_GOLD',
                        'price': gold_price,
                        'units': new_units
                    })
    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]['units'] * df.loc[(dates[-1], asset), 'Close']
        for asset in assets if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config['starting_cash']

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Export to CSV
        trade_df.to_csv("trade_log.csv", index=False)
        print("Trade log exported to 'trade_log.csv'")

    else:
        print("\n(No trades were executed during the period.)")

    return trade_df, final_value            

